# Example deck for running a discontinuity tolerant DIC analysis on a set of images
This jupyter notebook contains a typical DIC procedure for obtaining fine pass and a discontinuity tolerant second pass. 

In [ ]:
import os as o
from pathlib import Path
import time
import matplotlib.pyplot as plt
import crosspy as xpy

## Fill settings below

In [ ]:

# Define the folder path to images
folder_path=Path(r'C:\Users\alexb\Desktop\crosspy\data\Tom')

# NOTE THAT IMAGES SHOULD BE NUMBERED AND GRAYSCALE TIFFs ARE PREFERRED

# Coarse pass settings
coarse = 200 # in pixels
coarse_overlap = 70 # in percentage

# Fine pass settings
fine = 48 # in pixels
overlap = 80 # in percentage

# Heaviside pass settings
hs_size = 48
hs_overlap = 80
px_size = 30 # in nanometers

# fft filter settings: high pass, high pass width, low pass, low pass width
filter_settings=[4,2,15,8]

# Parallel settings

cores = 4 # number of CPU workers



## Here we run conventional DIC to compare with later.
The coarse pass settings will have to be changed manually by the user to find an ideal size and overlap. The coarse pass displacement field is used for image correction so it is important to use appropriate settings to get the largest amount of overlapping subsets for the second pass.

It is good practice to begin by changing the coarse pass size in the variable "coarse". The goal is to use the smallest "fine" size.

In [ ]:
if __name__ == "__main__":

    # build the dic class (but don't run it yet):
    Images = xpy.Imset(folder_path,'tif',[0,1])
    roi_1stpass = dict(size_pass = coarse, overlap_percentage = coarse_overlap, xcf_mesh=coarse)
    dic_1stpass=xpy.DIC(Images,roi_1stpass,filter_settings)

    # run the dic on specified images within the stack, and get displacements:
    dic_1stpass.run_sequential(cores=cores)
    dic_1stpass.plot_displacements()

    # correct the images and instantiate a new DIC class
    corrected_images=dic_1stpass.correct(method='polynomial',printing=1)
    
    # build second pass object
    roi_2ndpass = dict(size_pass = fine, overlap_percentage = overlap, xcf_mesh=fine)
    dic_2ndpass = xpy.DIC(corrected_images,roi_2ndpass,filter_settings,savingfolder=dic_1stpass.folder)


    # # run the second pass
    dic_2ndpass.run_sequential(cores=cores)
    dic_2ndpass.plot_displacements()

    dic_2ndpass.calculate_strain()
    dic_2ndpass.plot_strains()

    #dic_2ndpass.save_data()

## Here we run a third pass based on corrected images after first pass - this pass uses the heaviside algo

In [ ]:
if __name__ == "__main__":

    roi_3rdpass = dict(size_pass = hs_size, overlap_percentage = hs_overlap, xcf_mesh=hs_size)

    dic_3rdpass = xpy.DIC(corrected_images,roi_3rdpass,filter_settings,savingfolder=dic_1stpass.folder)

    # Run the third pass with hs=true
    dic_3rdpass.run_sequential(cores=cores, hs=True, cc_t=0., px_size=px_size)
    dic_3rdpass.plot_displacements()

    # Calculate strain
    dic_3rdpass.calculate_strain()
    dic_3rdpass.plot_strains()

## Here we plot the data in detail

In [ ]:
import cmocean # You may have to download this set of colormaps

cmap = cmocean.cm.deep_r


# XX strain data
pass_2_xx = dic_2ndpass.strain_11[:,:,0]
pass_3_xx = dic_3rdpass.strain_11[:,:,0]

# Heaviside data
r = dic_3rdpass.rd_maps[:,:,0]
t = dic_3rdpass.th_maps[:,:,0]
j = dic_3rdpass.rd_maps[:,:,0]


# Strain plot
fig1, axs = plt.subplots(1,2,figsize=(20,10))
im1 = axs[0].imshow(pass_2_xx,vmin=-0.05,vmax=0.05,cmap=cmap)
im2 = axs[1].imshow(pass_3_xx,vmin=-0.05,vmax=0.05,cmap=cmap)
fig1.colorbar(im1, ax=axs[0], shrink=0.8)
fig1.colorbar(im2, ax=axs[1], shrink=0.8)


# Heaviside plot
fig2, axs = plt.subplots(1,3,figsize=(30,10))
axs[0].imshow(r, cmap=cmap)
axs[1].imshow(t, cmap=cmap)
axs[2].imshow(j, cmap=cmap)


## Save data

In [ ]:

# This function is still a bit wonky
dic_3rdpass.save_data()


# Save images

fid1 = ""
fid2 = ""
fid = o.path.join(folder_path, fid + "." + "tif")

fig1.savefig(fid1, dpi=200)
fig2.savefig(fid2, dpi=200)